# HyLoRADA Experiments

**HyLoRADA**: Hybrid Low-Rank Direct Attention Adaptation

Novel contributions:
1. **Orthogonal initialization** - Prevents rank collapse
2. **Gated magnitude** - Learnable magnitude control
3. **Residual LoRA path** - Blends DoRA + LoRA dynamics

**Scripts:**
- `run_benchmark.py` - Compare all PEFT methods
- `optimize_hylorada.py` - Bayesian hyperparameter optimization

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'

In [ ]:
# Clone repo
!git clone https://github.com/SadiaTabassum1216/hylorada.git
%cd hylorada

In [ ]:
!pip install -q transformers datasets accelerate tqdm optuna

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## 1. Run Benchmark (Recommended)

Compare HyLoRADA against baselines (LoRA, DoRA, LoRaDA)

In [ ]:
# Compare all PEFT methods
!python run_benchmark.py \
    --model Qwen/Qwen2.5-0.5B \
    --max_length 1024 \
    --epochs 3 \
    --batch_size 1 \
    --grad_accum 16 \
    --num_train 1000 \
    --num_test 100 \
    --lora_rank 8 \
    --methods lora dora lorada hylorada \
    --output_dir ./benchmark_results

## 2. Hyperparameter Optimization (Optional)

Uses Bayesian optimization (Optuna TPE) to find best hyperparameters:
- `lora_rank`: [4, 8, 12, 16]
- `alpha_ratio`: [1.5, 3.0]
- `lora_plus_ratio`: [5, 20]
- `base_lr`: [1e-5, 5e-4]
- `gate_init`: [-1, 1]
- `residual_init`: [0, 0.5]

In [ ]:
# Run hyperparameter optimization (10 trials)
!python optimize_hylorada.py \
    --n_trials 10 \
    --epochs 2 \
    --model Qwen/Qwen2.5-0.5B

## 3. Optimized HyLoRADA Config

Use these optimized hyperparameters for best results:

In [ ]:
# Run with optimized hyperparameters
# After HPO, update these values from optimization_results.json

!python run_benchmark.py \
    --model Qwen/Qwen2.5-0.5B \
    --max_length 1024 \
    --epochs 3 \
    --batch_size 1 \
    --grad_accum 16 \
    --num_train 1000 \
    --num_test 100 \
    --lora_rank 12 \
    --methods dora hylorada \
    --output_dir ./optimized_results

In [ ]:
# View HPO results
import json
try:
    with open('optimization_results.json') as f:
        results = json.load(f)
    print("Best Hyperparameters:")
    for k, v in results['best_params'].items():
        print(f"  {k}: {v}")
    print(f"\nBest Loss: {results['best_value']:.4f}")
except:
    print("Run HPO first to see results")